RAG Pipelines- Data Ingestion to Vector DB Pipeline

In [1]:
import os
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pathlib import Path

/Users/nevintoms/Desktop/Projects/RAG_Langchain/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
### Read all the pdf's inside the directory
def process_all_pdfs(pdf_directory):
    """Process all PDF files in a directory"""
    all_documents = []
    pdf_dir = Path(pdf_directory)
    
    # Find all PDF files recursively
    pdf_files = list(pdf_dir.glob("**/*.pdf"))
    
    print(f"Found {len(pdf_files)} PDF files to process")
    
    for pdf_file in pdf_files:
        print(f"\nProcessing: {pdf_file.name}")
        try:
            loader = PyPDFLoader(str(pdf_file))
            documents = loader.load()
            
            # Add source information to metadata
            for doc in documents:
                doc.metadata['source_file'] = pdf_file.name
                doc.metadata['file_type'] = 'pdf'
            
            all_documents.extend(documents)
            print(f"  ✓ Loaded {len(documents)} pages")
            
        except Exception as e:
            print(f"  ✗ Error: {e}")
    
    print(f"\nTotal documents loaded: {len(all_documents)}")
    return all_documents

# Process all PDFs in the data directory
all_pdf_documents = process_all_pdfs("../data")

Found 3 PDF files to process

Processing: objectdetection.pdf
  ✓ Loaded 21 pages

Processing: embeddings.pdf
  ✓ Loaded 27 pages

Processing: attention.pdf
  ✓ Loaded 15 pages

Total documents loaded: 63


In [3]:
all_pdf_documents[0]

Document(metadata={'producer': 'pdfTeX-1.40.17', 'creator': 'LaTeX with hyperref package', 'creationdate': '2019-04-17T00:45:22+00:00', 'author': '', 'keywords': '', 'moddate': '2019-04-17T00:45:22+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.17 (TeX Live 2016) kpathsea version 6.2.2', 'subject': '', 'title': '', 'trapped': '/False', 'source': '../data/pdf/objectdetection.pdf', 'total_pages': 21, 'page': 0, 'page_label': '1', 'source_file': 'objectdetection.pdf', 'file_type': 'pdf'}, page_content='THIS PAPER HAS BEEN ACCEPTED BY IEEE TRANSACTIONS ON NEURAL NETWORKS AND LEARNING SYSTEMS FOR PUBLICATION 1\nObject Detection with Deep Learning: A Review\nZhong-Qiu Zhao, Member, IEEE, Peng Zheng,\nShou-tao Xu, and Xindong Wu, Fellow, IEEE\nAbstract—Due to object detection’s close relationship with\nvideo analysis and image understanding, it has attracted much\nresearch attention in recent years. Traditional object detection\nmethods are built on handcrafted featur

In [4]:
### Text splitting get into chunks

def split_documents(documents,chunk_size=1000,chunk_overlap=200):
    """Split documents into smaller chunks for better RAG performance"""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]
    )
    split_docs = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(split_docs)} chunks")
    
    # Show example of a chunk
    if split_docs:
        print(f"\nExample chunk:")
        print(f"Content: {split_docs[0].page_content[:1100]}...")
        print(f"Metadata: {split_docs[0].metadata}")
    
    return split_docs

In [5]:
chunks = split_documents(all_pdf_documents)
chunks

Split 63 documents into 356 chunks

Example chunk:
Content: THIS PAPER HAS BEEN ACCEPTED BY IEEE TRANSACTIONS ON NEURAL NETWORKS AND LEARNING SYSTEMS FOR PUBLICATION 1
Object Detection with Deep Learning: A Review
Zhong-Qiu Zhao, Member, IEEE, Peng Zheng,
Shou-tao Xu, and Xindong Wu, Fellow, IEEE
Abstract—Due to object detection’s close relationship with
video analysis and image understanding, it has attracted much
research attention in recent years. Traditional object detection
methods are built on handcrafted features and shallow trainable
architectures. Their performance easily stagnates by constructing
complex ensembles which combine multiple low-level image
features with high-level context from object detectors and scene
classiﬁers. With the rapid development in deep learning, more
powerful tools, which are able to learn semantic, high-level,
deeper features, are introduced to address the problems existing
in traditional architectures. These models behave differently
in network ar

[Document(metadata={'producer': 'pdfTeX-1.40.17', 'creator': 'LaTeX with hyperref package', 'creationdate': '2019-04-17T00:45:22+00:00', 'author': '', 'keywords': '', 'moddate': '2019-04-17T00:45:22+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.17 (TeX Live 2016) kpathsea version 6.2.2', 'subject': '', 'title': '', 'trapped': '/False', 'source': '../data/pdf/objectdetection.pdf', 'total_pages': 21, 'page': 0, 'page_label': '1', 'source_file': 'objectdetection.pdf', 'file_type': 'pdf'}, page_content='THIS PAPER HAS BEEN ACCEPTED BY IEEE TRANSACTIONS ON NEURAL NETWORKS AND LEARNING SYSTEMS FOR PUBLICATION 1\nObject Detection with Deep Learning: A Review\nZhong-Qiu Zhao, Member, IEEE, Peng Zheng,\nShou-tao Xu, and Xindong Wu, Fellow, IEEE\nAbstract—Due to object detection’s close relationship with\nvideo analysis and image understanding, it has attracted much\nresearch attention in recent years. Traditional object detection\nmethods are built on handcrafted featu

In [6]:
print(f"Content: {chunks[0].page_content[:2000]}...")

Content: THIS PAPER HAS BEEN ACCEPTED BY IEEE TRANSACTIONS ON NEURAL NETWORKS AND LEARNING SYSTEMS FOR PUBLICATION 1
Object Detection with Deep Learning: A Review
Zhong-Qiu Zhao, Member, IEEE, Peng Zheng,
Shou-tao Xu, and Xindong Wu, Fellow, IEEE
Abstract—Due to object detection’s close relationship with
video analysis and image understanding, it has attracted much
research attention in recent years. Traditional object detection
methods are built on handcrafted features and shallow trainable
architectures. Their performance easily stagnates by constructing
complex ensembles which combine multiple low-level image
features with high-level context from object detectors and scene
classiﬁers. With the rapid development in deep learning, more
powerful tools, which are able to learn semantic, high-level,
deeper features, are introduced to address the problems existing
in traditional architectures. These models behave differently
in network architecture, training strategy and optimization...


In [7]:
len(chunks[0].page_content[:1000])

986

embedding And vectorStoreDB

In [8]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity

In [17]:
model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = model.encode(["Nevin is good", "kevin is goopd"], show_progress_bar=True)

Batches: 100%|██████████| 1/1 [00:00<00:00,  1.88it/s]


In [27]:
embeddings[0]

array([-8.96324888e-02, -6.83456808e-02,  3.28811840e-03, -4.77464311e-02,
       -7.04665259e-02, -1.35691576e-02,  7.44805261e-02,  4.28414680e-02,
        1.87391154e-02,  3.16361487e-02, -2.51439940e-02, -4.87189814e-02,
       -2.84792781e-02,  6.90285340e-02, -2.67936382e-02, -3.48505639e-02,
        6.01727590e-02,  4.94053066e-02,  3.09734065e-02, -1.33773446e-01,
       -1.23999625e-01,  7.74960145e-02, -5.40082902e-02, -6.93457946e-02,
        7.34248664e-03, -8.94355252e-02,  2.93188952e-02, -1.29067758e-02,
       -4.13631368e-03, -6.51442260e-02,  2.11257748e-02, -7.71031305e-02,
       -5.21254055e-02, -3.25706936e-02, -3.45337205e-02,  5.81358420e-03,
       -5.67268655e-02,  2.37448327e-02, -3.98161747e-02,  3.43041196e-02,
       -1.34513564e-02, -1.29241534e-02, -1.50455674e-02, -6.27949163e-02,
        1.21100955e-02, -4.35552374e-02, -9.05472487e-02, -3.07407975e-02,
        1.27217263e-01,  1.92699768e-03, -8.20147023e-02,  4.48161736e-02,
       -8.78682919e-03,  

In [9]:
class EmbeddingManager:
    """Handles document embedding generation using SentenceTransformer"""
    
    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        """
        Initialize the embedding manager
        
        Args:
            model_name: HuggingFace model name for sentence embeddings
        """
        self.model_name = model_name
        self.model = None
        self._load_model()

    def _load_model(self):
        """Load the SentenceTransformer model"""
        try:
            print(f"Loading embedding model: {self.model_name}")
            self.model = SentenceTransformer(self.model_name)
            print(f"Model loaded successfully. Embedding dimension: {self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print(f"Error loading model {self.model_name}: {e}")
            raise

    def generate_embeddings(self, texts: List[str]) -> np.ndarray:
        """
        Generate embeddings for a list of texts
        
        Args:
            texts: List of text strings to embed
            
        Returns:
            numpy array of embeddings with shape (len(texts), embedding_dim)
        """
        if not self.model:
            raise ValueError("Model not loaded")
        
        print(f"Generating embeddings for {len(texts)} texts...")
        embeddings = self.model.encode(texts, show_progress_bar=True)
        print(f"Generated embeddings with shape: {embeddings.shape}")
        return embeddings


## initialize the embedding manager

embedding_manager=EmbeddingManager()
embedding_manager

Loading embedding model: all-MiniLM-L6-v2
Model loaded successfully. Embedding dimension: 384


VectorStore

In [10]:
class VectorStore:
    """Manages document embeddings in a ChromaDB vector store"""
    
    def __init__(self, collection_name: str = "pdf_documents", persist_directory: str = "../data/vector_store"):
        """
        Initialize the vector store
        
        Args:
            collection_name: Name of the ChromaDB collection
            persist_directory: Directory to persist the vector store
        """
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self._initialize_store()

    def _initialize_store(self):
        """Initialize ChromaDB client and collection"""
        try:
            # Create persistent ChromaDB client
            os.makedirs(self.persist_directory, exist_ok=True)
            self.client = chromadb.PersistentClient(path=self.persist_directory)
            
            # Get or create collection
            self.collection = self.client.get_or_create_collection(
                name=self.collection_name,
                metadata={"description": "PDF document embeddings for RAG"}
            )
            print(f"Vector store initialized. Collection: {self.collection_name}")
            print(f"Existing documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error initializing vector store: {e}")
            raise

    def add_documents(self, documents: List[Any], embeddings: np.ndarray):
        """
        Add documents and their embeddings to the vector store
        
        Args:
            documents: List of LangChain documents
            embeddings: Corresponding embeddings for the documents
        """
        if len(documents) != len(embeddings):
            raise ValueError("Number of documents must match number of embeddings")
        
        print(f"Adding {len(documents)} documents to vector store...")
        
        # Prepare data for ChromaDB
        ids = []
        metadatas = []
        documents_text = []
        embeddings_list = []
        
        for i, (doc, embedding) in enumerate(zip(documents, embeddings)):
            # Generate unique ID
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)
            
            # Prepare metadata
            metadata = dict(doc.metadata)
            metadata['doc_index'] = i
            metadata['content_length'] = len(doc.page_content)
            metadatas.append(metadata)
            
            # Document content
            documents_text.append(doc.page_content)
            
            # Embedding
            embeddings_list.append(embedding.tolist())
        
        # Add to collection
        try:
            self.collection.add(
                ids=ids,
                embeddings=embeddings_list,
                metadatas=metadatas,
                documents=documents_text
            )
            print(f"Successfully added {len(documents)} documents to vector store")
            print(f"Total documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error adding documents to vector store: {e}")
            raise

vectorstore=VectorStore()
vectorstore
    

Vector store initialized. Collection: pdf_documents
Existing documents in collection: 0


In [11]:
chunks


[Document(metadata={'producer': 'pdfTeX-1.40.17', 'creator': 'LaTeX with hyperref package', 'creationdate': '2019-04-17T00:45:22+00:00', 'author': '', 'keywords': '', 'moddate': '2019-04-17T00:45:22+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.17 (TeX Live 2016) kpathsea version 6.2.2', 'subject': '', 'title': '', 'trapped': '/False', 'source': '../data/pdf/objectdetection.pdf', 'total_pages': 21, 'page': 0, 'page_label': '1', 'source_file': 'objectdetection.pdf', 'file_type': 'pdf'}, page_content='THIS PAPER HAS BEEN ACCEPTED BY IEEE TRANSACTIONS ON NEURAL NETWORKS AND LEARNING SYSTEMS FOR PUBLICATION 1\nObject Detection with Deep Learning: A Review\nZhong-Qiu Zhao, Member, IEEE, Peng Zheng,\nShou-tao Xu, and Xindong Wu, Fellow, IEEE\nAbstract—Due to object detection’s close relationship with\nvideo analysis and image understanding, it has attracted much\nresearch attention in recent years. Traditional object detection\nmethods are built on handcrafted featu

In [12]:
### Convert the text to embeddings
texts=[doc.page_content for doc in chunks]

## Generate the Embeddings

embeddings=embedding_manager.generate_embeddings(texts)

##store int he vector dtaabase
vectorstore.add_documents(chunks,embeddings)

Generating embeddings for 356 texts...


Batches: 100%|██████████| 12/12 [00:03<00:00,  3.33it/s]

Generated embeddings with shape: (356, 384)
Adding 356 documents to vector store...
Successfully added 356 documents to vector store
Total documents in collection: 356


Retriever Pipeline From VectorStore

In [13]:
class RAGRetriever:
    """Handles query-based retrieval from the vector store"""
    
    def __init__(self, vector_store: VectorStore, embedding_manager: EmbeddingManager):
        """
        Initialize the retriever
        
        Args:
            vector_store: Vector store containing document embeddings
            embedding_manager: Manager for generating query embeddings
        """
        self.vector_store = vector_store
        self.embedding_manager = embedding_manager

    def retrieve(self, query: str, top_k: int = 5, score_threshold: float = 0.0) -> List[Dict[str, Any]]:
        """
        Retrieve relevant documents for a query
        
        Args:
            query: The search query
            top_k: Number of top results to return
            score_threshold: Minimum similarity score threshold
            
        Returns:
            List of dictionaries containing retrieved documents and metadata
        """
        print(f"Retrieving documents for query: '{query}'")
        print(f"Top K: {top_k}, Score threshold: {score_threshold}")
        
        # Generate query embedding
        query_embedding = self.embedding_manager.generate_embeddings([query])[0]
        
        # Search in vector store
        try:
            results = self.vector_store.collection.query(
                query_embeddings=[query_embedding.tolist()],
                n_results=top_k
            )
            
            # Process results
            retrieved_docs = []
            
            if results['documents'] and results['documents'][0]:
                documents = results['documents'][0]
                metadatas = results['metadatas'][0]
                distances = results['distances'][0]
                ids = results['ids'][0]
                
                for i, (doc_id, document, metadata, distance) in enumerate(zip(ids, documents, metadatas, distances)):
                    # Convert distance to similarity score (ChromaDB uses cosine distance)
                    similarity_score = 1 - distance
                    
                    if similarity_score >= score_threshold:
                        retrieved_docs.append({
                            'id': doc_id,
                            'content': document,
                            'metadata': metadata,
                            'similarity_score': similarity_score,
                            'distance': distance,
                            'rank': i + 1
                        })
                
                print(f"Retrieved {len(retrieved_docs)} documents (after filtering)")
            else:
                print("No documents found")
            
            return retrieved_docs
            
        except Exception as e:
            print(f"Error during retrieval: {e}")
            return []

rag_retriever=RAGRetriever(vectorstore,embedding_manager)

In [14]:
rag_retriever

In [15]:
rag_retriever.retrieve("What is attention is all you need")

Retrieving documents for query: 'What is attention is all you need'
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:01<00:00,  1.22s/it]

Generated embeddings with shape: (1, 384)
Retrieved 1 documents (after filtering)


[{'id': 'doc_200c8466_316',
  'content': '3.2 Attention\nAn attention function can be described as mapping a query and a set of key-value pairs to an output,\nwhere the query, keys, values, and output are all vectors. The output is computed as a weighted sum\n3',
  'metadata': {'keywords': '',
   'file_type': 'pdf',
   'page': 2,
   'moddate': '2024-04-10T21:11:43+00:00',
   'title': '',
   'source': '../data/pdf/attention.pdf',
   'doc_index': 316,
   'trapped': '/False',
   'source_file': 'attention.pdf',
   'total_pages': 15,
   'producer': 'pdfTeX-1.40.25',
   'creationdate': '2024-04-10T21:11:43+00:00',
   'creator': 'LaTeX with hyperref',
   'content_length': 216,
   'page_label': '3',
   'author': '',
   'subject': '',
   'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5'},
  'similarity_score': 0.13995516300201416,
  'distance': 0.8600448369979858,
  'rank': 1}]

RAG Pipeline- VectorDB To LLM Output Generation

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

print(os.getenv("GROQ_API_KEY"))